Note: This notebook is currently for generating fully-developed seniors, not recruits

(To convert to underclassmen, take X% of current Skills?

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
def clamp(v, n, x):
    return max(min(v, x), n)

In [3]:
firsts = pd.read_csv("yob1998.txt", header=None)
firsts.columns = ["Name", "Sex", "Count"]
firsts = firsts.loc[firsts.Sex=="M"].reset_index(drop=True)
firsts["Frequency"] = firsts.Count / firsts.Count.sum()
firsts

,Name,Sex,Count,Frequency
0,Michael,M,36615,0.019161
1,Jacob,M,36016,0.018848
2,Matthew,M,31142,0.016297
3,Joshua,M,28089,0.014699
4,Christopher,M,27002,0.014131
...,...,...,...,...
11298,Zayvon,M,5,0.000003
11299,Ziaire,M,5,0.000003
11300,Zildjian,M,5,0.000003
11301,Zykeem,M,5,0.000003


In [4]:
lasts = pd.read_excel("Surnames_2010Census_Top1000.xlsx")
lasts.columns = lasts.loc[1]
lasts = lasts[2:1002][["SURNAME", "PROPORTION PER 100,000 POPULATION"]].reset_index(drop=True)
lasts.columns = ["Surname", "Proportion"]
lasts["Surname"] = lasts.Surname.map(lambda x: x.lower().capitalize())
lasts["Proportion"] = lasts["Proportion"] / lasts["Proportion"].sum()
lasts

,Surname,Proportion
0,Smith,0.0202154
1,Johnson,0.0159938
2,Williams,0.0134487
3,Brown,0.0118911
4,Jones,0.0117955
...,...,...
995,Odom,0.000289736
996,Daugherty,0.000289492
997,Travis,0.000289492
998,Tang,0.000289248


In [5]:
#np.random.choice(firsts.Name, size=10, p=firsts.Frequency)

In [6]:
players = pd.DataFrame(columns=["First", "Last", "Height", "Weight",
                                "T_Strength", "T_Athleticism", "T_Dexterity", "T_IQ", "T_Personality", "T_Arm", "T_Leg",])

In [7]:
# returns a row as a dict
def GeneratePlayer(pBonus):
    result = {
        "First": np.random.choice(firsts.Name, p=firsts.Frequency),
        "Last": np.random.choice(lasts.Surname, p=lasts.Proportion)
    }
    
    result["Height"] = 68 + np.random.normal(5, 2)
    H = result["Height"] - 68.0
    result["Weight"] = np.random.normal(165 + H**2, (10.0/3.0)*H + 12)
    if result["Weight"] >= 380:
        return "error"
    elif result["Weight"] < (155 + 15*(H/3)):
        return "error"
    W = (result["Weight"] - 150)/2
    H = 100 * (H/12)
    
    tBonus = np.random.normal(0, 6)
    #result["T_Strength"] = np.random.normal(np.average((50, W), weights=(2, 1)), 10)
    result["T_Strength"] = np.random.normal(np.mean((50, W)), 10)
    result["T_Athleticism"] = np.random.normal(np.mean((50, 100-W)), 10)
    result["T_Dexterity"] = np.random.normal(np.mean((50, 100-W)), 15)
    result["T_IQ"] = np.random.normal(50, 12)
    result["T_Personality"] = np.random.normal(50, 12)
    result["T_Arm"] = np.random.normal(45, 15)
    result["T_Leg"] = np.random.normal(45, 15)
    
    for t in [c for c in players.columns if c[:2] == "T_"]:
        result[t] = clamp(result[t], 10, 120)
    
    pBonus = (pBonus * result["T_Personality"])/100
    pBonus = clamp(pBonus, 0, 100)
            
    result["S_ArmStr"] = 0.7*result["T_Arm"] + 0.1*result["T_Strength"] + 0.05*np.random.uniform(0, 100) + 0.15*pBonus
    result["S_ArmAccShort"] = 0.65*result["T_Arm"] + 0.15*result["T_Dexterity"] + 0.1*np.random.uniform(0, 100) + 0.1*pBonus
    result["S_ArmAccMed"] = 0.65*result["T_Arm"] + 0.1*result["T_Dexterity"] + 0.05*result["T_Strength"] + 0.1*np.random.uniform(0, 100) + 0.1*pBonus
    result["S_ArmAccLong"] = 0.7*result["T_Arm"] + 0.1*result["T_Strength"] + 0.05*result["T_Dexterity"] + 0.05*np.random.uniform(0, 100) + 0.1*pBonus
    
    result["S_ReadDefPass"] = 0.6*result["T_IQ"] + 0.1*H + 0.05*np.random.uniform(0, 100) + 0.25*pBonus
    result["S_ReadDefRun"] = 0.7*result["T_IQ"] + 0.2*np.random.uniform(0, 100) + 0.1*pBonus
    
    result["S_Speed"] = 0.9*result["T_Athleticism"] + 0.1*np.random.uniform(0, 100)
    result["S_Acceleration"] = 0.8*result["T_Athleticism"] + 0.1*result["T_Dexterity"] + 0.1*np.random.uniform(0, 100)
    result["S_Agility"] = 0.8*result["T_Athleticism"] + 0.1*result["T_Dexterity"] + 0.1*np.random.uniform(0, 100)
    result["S_RunPower"] = 0.6*result["T_Strength"] + 0.2*result["T_Athleticism"] + 0.1*W +0.1*np.random.uniform(0, 100)
    
    result["S_Vertical"] = 0.5*result["T_Athleticism"] + 0.4*H + 0.1*np.random.uniform(0, 100)
    result["S_Hands"] = 0.7*result["T_Dexterity"] + 0.2*np.random.uniform(0, 100) + 0.1*pBonus
    
    result["S_RunBlock"] = 0.7*result["T_Strength"] + 0.1*W + 0.1*np.random.uniform(0, 100) + 0.1*pBonus
    result["S_PassBlock"] = 0.5*result["T_Strength"] + 0.2*result["T_Athleticism"] + 0.1*W + 0.1*np.random.uniform(0, 100) + 0.1*pBonus
    
    result["S_TackApproach"] = 0.7*result["T_Athleticism"] + 0.1*result["T_Dexterity"] + 0.2*np.random.uniform(0, 100)
    result["S_TackBringDown"] = 0.6*result["T_Strength"] + 0.2*result["T_Athleticism"] + 0.1*W + 0.1*np.random.uniform(0, 100)
    
    result["S_ShedBlockStr"] = 0.6*result["T_Strength"] + 0.2*result["T_Athleticism"] + 0.1*W + 0.1*np.random.uniform(0, 100)
    result["S_ShedBlockAgl"] = 0.4*result["T_Athleticism"] + 0.35*result["T_Strength"] + 0.1*result["T_Dexterity"] + 0.1*np.random.uniform(0, 100) + 0.05*pBonus
    
    result["S_CoverMan"] = 0.7*result["T_Athleticism"] + 0.1*result["T_Dexterity"] + 0.1*np.random.uniform(0, 100) + 0.1*pBonus
    result["S_CoverZone"] = 0.5*result["T_Athleticism"] + 0.2*result["T_IQ"] + 0.1*result["T_Dexterity"] + 0.1*np.random.uniform(0, 100) + 0.1*pBonus
    result["S_ReadOff"] = 0.6*result["T_IQ"] + 0.1*H + 0.1*np.random.uniform(0, 100) + 0.2*pBonus
    
    result["S_KickPow"] = 0.9*result["T_Leg"] + 0.1*np.random.uniform(0, 100)
    result["S_KickAcc"] = 0.8*result["T_Leg"] + 0.1*result["T_Dexterity"] + np.random.uniform(0, 100)
    
    result["S_PuntPow"] = 0.9*result["T_Leg"] + 0.1*np.random.uniform(0, 100)
    result["S_PuntAcc"] = 0.8*result["T_Leg"] + 0.1*result["T_Dexterity"] + np.random.uniform(0, 100)
    
    return result

In [8]:
rowList = []
for i in range(25000):
    p = GeneratePlayer(pBonus=50)
    while p == "error":
        p = GeneratePlayer(pBonus=50)
    
    rowList.append(p)

players = pd.DataFrame(rowList)
players.round(1)

,First,Last,Height,Weight,T_Strength,T_Athleticism,T_Dexterity,T_IQ,T_Personality,T_Arm,T_Leg,S_ArmStr,S_ArmAccShort,S_ArmAccMed,S_ArmAccLong,S_ReadDefPass,S_ReadDefRun,S_Speed,S_Acceleration,S_Agility,S_RunPower,S_Vertical,S_Hands,S_RunBlock,S_PassBlock,S_TackApproach,S_TackBringDown,S_ShedBlockStr,S_ShedBlockAgl,S_CoverMan,S_CoverZone,S_ReadOff,S_KickPow,S_KickAcc,S_PuntPow,S_PuntAcc
0,Andrew,Villa,73.3,225.5,48.0,62.2,70.0,52.0,60.0,65.0,49.6,57.2,59.5,56.2,60.2,47.4,53.5,63.0,66.4,61.8,50.2,53.8,66.0,44.7,48.2,63.8,49.9,51.9,58.0,58.3,54.4,46.8,45.6,88.9,49.5,103.2
1,Rashaud,Martinez,70.3,170.2,17.4,80.3,50.0,36.3,66.0,44.4,62.6,39.8,49.5,42.0,39.1,35.0,42.4,79.8,69.8,73.0,32.0,57.3,57.2,25.2,31.3,65.1,28.2,34.3,54.1,73.1,62.5,38.1,64.2,120.9,61.9,98.8
2,Nathaniel,Davis,71.8,202.8,22.5,64.8,46.8,69.9,44.5,67.8,24.4,54.4,60.5,52.3,56.2,50.8,67.0,62.9,61.1,61.4,37.0,48.2,36.3,20.9,34.5,61.1,34.8,34.2,40.1,61.3,54.8,57.2,23.5,71.5,25.9,57.4
3,Austin,Hudson,73.7,229.7,39.5,57.1,40.5,59.8,31.8,54.7,31.9,45.9,47.1,47.2,50.6,48.9,61.7,57.6,51.3,53.4,44.3,53.7,34.5,38.6,41.2,55.2,39.4,47.8,46.2,50.5,48.4,50.4,33.6,59.5,35.6,99.4
4,Jonathan,Dickerson,70.1,173.2,42.6,59.7,90.6,57.7,47.6,21.7,49.0,25.9,30.7,29.2,29.1,45.0,44.1,60.4,57.9,66.3,47.9,45.9,81.5,34.0,42.2,56.7,45.6,39.7,52.0,61.8,59.2,43.2,50.5,118.5,51.7,146.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,James,Villegas,73.5,260.1,68.5,43.0,28.5,14.7,69.3,56.8,25.0,55.0,54.6,47.7,54.7,24.4,24.8,43.6,47.2,41.0,61.7,46.4,36.1,58.5,57.2,51.6,57.9,62.5,46.5,44.6,33.0,25.2,25.4,117.3,24.2,37.1
24996,Charles,Mclaughlin,75.2,215.7,41.4,49.8,60.8,69.0,33.7,27.7,49.2,29.9,29.3,29.5,29.3,56.4,63.5,49.9,48.4,47.5,43.1,58.5,58.3,40.2,42.1,58.1,41.3,47.3,45.5,51.6,47.2,55.6,48.6,55.7,52.6,92.8
24997,Christopher,Warren,74.2,189.6,33.1,63.8,78.3,62.4,37.3,47.5,67.5,40.8,48.0,47.9,44.6,52.0,58.3,63.1,64.9,59.4,38.3,55.1,59.6,31.6,38.2,61.2,41.6,41.6,53.5,63.3,58.9,46.8,69.6,93.2,62.7,90.0
24998,Cameron,Hill,75.3,248.7,42.7,34.3,58.0,29.2,38.8,31.1,46.9,31.3,37.1,39.8,31.8,31.5,31.3,39.0,35.2,34.5,38.9,48.0,59.4,38.9,44.9,35.9,41.2,42.6,40.0,32.5,40.0,32.9,42.6,94.8,46.0,74.4


In [9]:
# standardize to N(50, 12)
for c in [c for c in players.columns if "S_" in c]:
    ss = StandardScaler()
    players[c] = ss.fit_transform(players[c].to_numpy().reshape(-1, 1))
    players[c] = players[c] * 12 + 50

players

,First,Last,Height,Weight,T_Strength,T_Athleticism,T_Dexterity,T_IQ,T_Personality,T_Arm,T_Leg,S_ArmStr,S_ArmAccShort,S_ArmAccMed,S_ArmAccLong,S_ReadDefPass,S_ReadDefRun,S_Speed,S_Acceleration,S_Agility,S_RunPower,S_Vertical,S_Hands,S_RunBlock,S_PassBlock,S_TackApproach,S_TackBringDown,S_ShedBlockStr,S_ShedBlockAgl,S_CoverMan,S_CoverZone,S_ReadOff,S_KickPow,S_KickAcc,S_PuntPow,S_PuntAcc
0,Andrew,Villa,73.329465,225.518181,47.954142,62.181207,69.953896,51.975204,59.961366,64.975402,49.590992,67.534603,66.000625,63.551519,69.048817,56.701668,56.966461,54.366286,58.345577,53.279437,58.707909,52.415689,60.912917,56.940152,59.122020,56.468082,58.170689,61.141121,64.728949,53.637571,51.583034,53.591480,50.039214,48.820999,53.450567,54.285820
1,Rashaud,Martinez,70.349948,170.201696,17.381758,80.337198,50.031219,36.328263,65.965118,44.381192,62.594783,47.808422,54.532325,46.882967,45.104916,37.405651,43.952549,71.638345,62.076360,65.501306,32.278905,57.934881,52.802197,33.972915,30.996167,57.941775,26.628119,35.709697,57.379291,71.761383,63.906247,40.568902,66.224038,61.069794,64.279548,52.606805
2,Nathaniel,Davis,71.772565,202.816085,22.486975,64.808609,46.794369,69.940635,44.506366,67.830225,24.394538,64.360328,67.220004,58.985838,64.429090,62.049308,72.677879,54.184182,52.542824,52.849687,39.518913,43.677499,33.709566,28.944551,36.403506,53.554189,36.314579,35.572603,30.910395,57.326814,52.215547,69.100320,30.770603,42.181322,32.890440,36.730337
3,Austin,Hudson,73.666186,229.677414,39.475774,57.076837,40.496676,59.809063,31.804710,54.653463,31.899638,54.691483,51.702161,52.954908,58.071729,59.014061,66.562910,48.740703,41.938500,44.144276,50.141632,52.268147,32.025135,49.750349,47.603648,47.115507,42.901553,55.287491,42.381784,44.171352,42.459690,58.906325,39.559153,37.604095,41.328974,52.861893
4,Jonathan,Dickerson,70.065086,173.210995,42.581242,59.720136,90.583542,57.668770,47.624509,21.698522,48.952261,32.099457,32.828730,31.825916,33.718923,52.974498,45.883329,51.629389,49.115837,58.192187,55.300713,40.121410,75.093697,44.353966,49.114251,48.778654,51.915281,43.453489,53.425903,57.947965,58.825818,48.253241,54.307794,60.154723,55.380031,70.982829
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,James,Villegas,73.528098,260.056591,68.450965,43.036478,28.460610,14.652134,69.312980,56.820823,24.980853,65.059855,60.398699,53.509076,62.815595,20.777118,23.261105,34.407259,37.370895,30.683378,75.389415,40.868415,33.473696,73.180528,74.105700,43.191264,69.815327,76.521383,42.896424,36.909524,19.068594,21.365217,32.420844,59.679929,31.353905,28.946061
24996,Charles,Mclaughlin,75.167720,215.683638,41.401476,49.821533,60.779464,69.032819,33.702009,27.714952,49.174029,36.583723,31.184250,32.161249,33.970026,70.687765,68.587704,40.875556,38.759625,37.745083,48.441774,59.833229,53.846984,51.649235,49.038243,50.251165,45.665446,54.517045,41.154598,45.493886,40.667098,66.652982,52.627091,36.162397,56.137725,50.314589
24997,Christopher,Warren,74.236642,189.573325,33.102708,63.845046,78.267603,62.412077,37.345034,47.463082,67.464953,48.973888,52.791479,53.814607,51.316366,63.870079,62.528378,54.438680,56.724506,50.675014,41.466785,54.510466,55.012058,41.482646,42.488711,53.715848,46.135612,46.321489,56.223161,59.756441,58.410008,53.628519,70.947309,50.466250,64.908047,49.224179
24998,Cameron,Hill,75.332693,248.734379,42.736291,34.309655,57.992255,29.220614,38.845722,31.121252,46.937386,38.203765,40.155246,44.247748,36.754676,31.932569,30.869864,29.626395,24.272350,23.604006,42.304604,43.417753,54.781448,50.123714,53.620327,26.063687,45.519930,47.771636,30.598028,22.217136,29.718353,32.899007,47.406266,51.081596,50.402523,43.252071


In [10]:
# output to CSV file for drafting

players.to_csv("players.csv", index=False)